In [1]:
%load_ext autoreload
%autoreload 2

Download if file is not in folder.

In [2]:
from litedict import SQLDict

In [3]:
TEST_1 = "key_test_1"
TEST_2 = "key_test_2"

Basic functionality

In [4]:
d = SQLDict(":memory:")

d[TEST_1] = "asdfoobar"

assert d[TEST_1] == "asdfoobar"

del d[TEST_1]

assert d.get(TEST_1, None) is None

Glob matching

In [5]:
d[TEST_1] = "asdfoobar"

d[TEST_2] = "foobarasd"

d["key_testx_3"] = "barasdfoo"

assert d.glob("key_test*") == ["asdfoobar", "foobarasd", "barasdfoo"]

assert d.glob("key_test_?") == ["asdfoobar", "foobarasd"]

assert d.glob("key_tes[tx]*") == ["asdfoobar", "foobarasd", "barasdfoo"]

Numbers

In [6]:
d[TEST_1] = 1

d[TEST_2] = 2

assert d[TEST_1] + d[TEST_2] == 3

In [7]:
with d.transaction():
    d["asd"] = "efg"
    d["foo"] = "bar"
    assert d.conn.in_transaction

In [8]:
try:
    with d.transaction():
        d["failed"] = "no"

        assert d.conn.in_transaction

        raise Exception
except:
    # check the transaction succesfully rolled back
    assert d.get("failed") is None

In [9]:
for k, v in d.items():
    print(k, v)

key_testx_3 barasdfoo
key_test_1 1
key_test_2 2
asd efg
foo bar


Test moving from/to disk/memory.

In [10]:
import os
import pickle

fname = "test_disk.db"

d = SQLDict(
    ":memory:",
    encoder=lambda x: pickle.dumps(x).hex(),
    decoder=lambda x: pickle.loads(bytes.fromhex(x)),
)

d["foo"] = "bar"
d["asd"] = 2

d.to_disk("test_disk.db")

assert fname in os.listdir()
assert "foo" in d
assert "asd" in d

d.close()
del d


d = SQLDict(
    fname,
    encoder=lambda x: pickle.dumps(x).hex(),
    decoder=lambda x: pickle.loads(bytes.fromhex(x)),
)

assert d["foo"] == "bar"
assert d["asd"] == 2

d.to_memory()

os.unlink(fname)

assert d["foo"] == "bar"
assert d["asd"] == 2

## Benchmarks

In [11]:
from string import ascii_lowercase, printable
from random import choice
import random


def random_string(string_length=10, fuzz=False, space=False):
    """Generate a random string of fixed length """
    letters = ascii_lowercase
    letters = letters + " " if space else letters
    if fuzz:
        letters = printable
    return "".join(choice(letters) for i in range(string_length))

In [12]:
import gc

import pickle

import json

**Pickle**

In [13]:
d = SQLDict(
    ":memory:",
    encoder=lambda x: pickle.dumps(x).hex(),
    decoder=lambda x: pickle.loads(bytes.fromhex(x)),
)

In [14]:
gc.collect()

67

In [15]:
%%timeit -n20000 -r10

d[random_string(8)] = random_string(50)

d.get(random_string(8), None)

44.2 µs ± 431 ns per loop (mean ± std. dev. of 10 runs, 20,000 loops each)


**Noop**

In [16]:
d = SQLDict(
    ":memory:",
    encoder=lambda x: x,
    decoder=lambda x: x,
)

In [17]:
gc.collect()

3

In [18]:
%%timeit -n20000 -r10

d[random_string(8)] = random_string(50)

d.get(random_string(8), None)

42.4 µs ± 750 ns per loop (mean ± std. dev. of 10 runs, 20,000 loops each)


**JSON**

In [19]:
d = SQLDict(
    ":memory:",
    encoder=lambda x: json.dumps(x),
    decoder=lambda x: json.loads(x),
)

In [20]:
gc.collect()

3

In [21]:
%%timeit -n20000 -r10

d[random_string(8)] = random_string(50)

d.get(random_string(8), None)

44.1 µs ± 949 ns per loop (mean ± std. dev. of 10 runs, 20,000 loops each)


**Pickle Python obj**

In [22]:
d = SQLDict(
    ":memory:",
    encoder=lambda x: pickle.dumps(x).hex(),
    decoder=lambda x: pickle.loads(bytes.fromhex(x)),
)

In [23]:
gc.collect()

3

In [24]:
class C:
    def __init__(self, x):
        self.x = x

    def pp(self):
        return x

    def f(self):
        def _f(y):
            return y * self.x ** 2

        return _f

In [25]:
%%timeit -n20000 -r10

d[random_string(8)] = C(random.randint(1, 200))

d.get(random_string(8), None)

29.3 µs ± 635 ns per loop (mean ± std. dev. of 10 runs, 20,000 loops each)


**Dictionary**

In [26]:
d = {}

In [27]:
gc.collect()

3

In [28]:
%%timeit -n20000 -r10

d[random_string(8)] = random_string(50)

d.get(random_string(8), None)

30.6 µs ± 1.03 µs per loop (mean ± std. dev. of 10 runs, 20,000 loops each)


## Writeback Cache

### Missing

In [29]:
d = SQLDict(":memory:",writeback=False)
mylist = d['test'] = []

assert not d['test'] is mylist, "using writeback when not set"
assert mylist == d['test'], "equality should still pass"

mylist.append(1)
assert mylist != d['test'], "Should not be equal"
d.close()

### With writeback

In [30]:
try:
    d = SQLDict("__tmp.db",writeback=True)
    mylist = d['test'] = []

    assert mylist is d['test'], "should be the same object"
    assert mylist == d['test'], "equality should pass"

    mylist.append(1)
    assert mylist is d['test'], "should be the same object"
    assert mylist == d['test'], "Should still be equal"
    
    d2 =  SQLDict("__tmp.db",writeback=True)
    
    # This should fail since there was no sync
    assert d2['test'] != d['test']
    
    d.sync()
    
    d2 =  SQLDict("__tmp.db",writeback=True)
    # Now this should pass
    print(f"{d2['test'] = } {d['test'] = }")
    assert d2['test'] == d['test']
    
finally:
    pass#os.unlink("__tmp.db")

d2['test'] = [1] d['test'] = [1]


These are a few more methods where the code was modified for writeback

In [31]:
for writeback in [True,False]:
    d = SQLDict(":memory:",writeback=False)
    mylist = d['test'] = ['this','is','a','test']

    assert d.glob('te*') == [['this', 'is', 'a', 'test']]
    
    del d['test']
    assert 'test' not in d
